In [2]:
import numpy as np
import math
import pandas as pd
from geopy import distance
from itertools import combinations
import folium
import ast
import time
#import timeit
# start_time=time.time()
# print(time.time()-start_time)

In [3]:
file_path='/home/dataplus-01/Documents/Modules_Socle/Python/Projet_arbre/data/trees.csv'
data = pd.read_csv(file_path, sep=",", header = 0, index_col=False,names=None)
# on peut également appliquer un converter au moment de read, ou encore de read qu'une colonne
# df = pd.read_csv('/home/dataplus-01/Documents/Modules_Socle/Python/Projet_arbre/data/trees copy.csv'
#                  , converters={'GeoJSON': pd.read_json})

In [4]:
# on identifie la colonne qui nous intéresse
data = pd.DataFrame(data = data)
GeoJSON=data["GeoJSON"]
GeoJSON

0        {"type":"Point","coordinates":[5.7603469008942...
1        {"type":"Point","coordinates":[5.7598264646441...
2        {"type":"Point","coordinates":[5.7599807314486...
3        {"type":"Point","coordinates":[5.7600570301267...
4        {"type":"Point","coordinates":[5.7600202792924...
                               ...                        
31238    {"type":"Point","coordinates":[5.7117169490564...
31239    {"type":"Point","coordinates":[5.7115904446110...
31240    {"type":"Point","coordinates":[5.7114873970721...
31241    {"type":"Point","coordinates":[5.7117859768817...
31242    {"type":"Point","coordinates":[5.7113357012675...
Name: GeoJSON, Length: 31243, dtype: object

In [5]:
# on la reformate comme il faut
GeoJSON_dict=GeoJSON.map(lambda a:ast.literal_eval(a))
type(GeoJSON_dict.iloc[0])

dict

In [6]:
# test d'accés à l'information longitude/latitude
GeoJSON_dict[0]["coordinates"][0]

5.76034690089422

In [7]:
# pour la suite on travaillera sur un échantillon aléatoire de 500 arbres
GeoJSON_dict=GeoJSON_dict.sample(500)

In [24]:
# Mettez les arbres sur une carte en utilisant Folium.
m = folium.Map(location=[45.16667, 5.71667], zoom_start=12, tiles='Stamen Toner')
feature_group = folium.FeatureGroup("Arbres")
for ligne in range(0,500):
    feature_group.add_child(folium
                            .CircleMarker(location=[
                                float(GeoJSON_dict.iloc[ligne]["coordinates"][1]),
                                float(GeoJSON_dict.iloc[ligne]["coordinates"][0])],
                                         radius=2)) 
m.add_child(feature_group)
m

In [9]:
#Calculer la distance entre deux arbres de deux manières différentes
#avec la fct distance de geopy
a=(GeoJSON_dict.iloc[0]["coordinates"][0],GeoJSON_dict.iloc[0]["coordinates"][1])
b=(GeoJSON_dict.iloc[1]["coordinates"][0],GeoJSON_dict.iloc[1]["coordinates"][1])
print(a,b)
distance.distance(a, b).meters


(5.74557121291162, 45.1883950991829) (5.73984876257375, 45.1897349392367)


649.988448944327

In [10]:
# avec la formule d'Haverstine
def dist_haversine(lat1,lon1,lat2,lon2):
    R = 6371 # Radius of the earth in km
    dLat = deg2rad(lat2-lat1)
    dLon = deg2rad(lon2-lon1)
    a = (np.sin(dLat/2) * np.sin(dLat/2) +
    np.cos(deg2rad(lat1)) * np.cos(deg2rad(lat2)) *
    np.sin(dLon/2) * np.sin(dLon/2))
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R * c # Distance in km
    return d

def deg2rad(deg):
    return deg*(np.pi/180)

print(dist_haversine(a[0], a[1], b[0], b[1]))

0.653345995702908


In [11]:
# on convertie le dictionnaire en series

# data_lat_long=pd.DataFrame()
# data_lat_long["latitude"]=GeoJSON_dict.apply(lambda x: x["coordinates"][0])
# data_lat_long["longitude"]=GeoJSON_dict.apply(lambda x: x["coordinates"][1])
# data_lat_long

# ou plus simplement
azerty=GeoJSON_dict.apply(pd.Series)
azerty
# on pourrait extraire uniquement les longitudes et latitudes
# azer=azerty["coordinates"].apply(pd.Series)

,type,coordinates
13604,Point,"[5.74557121291162, 45.1883950991829]"
3492,Point,"[5.73984876257375, 45.1897349392367]"
23274,Point,"[5.71755493262181, 45.175784450631]"
20823,Point,"[5.73657286272301, 45.1947469025278]"
6014,Point,"[5.72876386133335, 45.1616942436905]"
...,...,...
8965,Point,"[5.7247036113896, 45.1758442284526]"
16124,Point,"[5.70928006860134, 45.1682021368581]"
8607,Point,"[5.70974403260293, 45.1751742813924]"
16842,Point,"[5.71848743575016, 45.1769031442266]"


In [12]:
# test de fct lambda pour réaliser des tuples de coordonnées
azerty["coordinates"].apply(lambda x: tuple(x))

13604    (5.74557121291162, 45.1883950991829)
3492     (5.73984876257375, 45.1897349392367)
23274     (5.71755493262181, 45.175784450631)
20823    (5.73657286272301, 45.1947469025278)
6014     (5.72876386133335, 45.1616942436905)
                         ...                 
8965      (5.7247036113896, 45.1758442284526)
16124    (5.70928006860134, 45.1682021368581)
8607     (5.70974403260293, 45.1751742813924)
16842    (5.71848743575016, 45.1769031442266)
10741    (5.73939737746003, 45.1610056163133)
Name: coordinates, Length: 500, dtype: object

In [13]:
# Quels sont les 10 arbres les plus proches de chez vous ?
# faire un dataframe avec mes coordonnées
# myHome=pd.DataFrame(data=[[5.720382352069388,45.18841421604648]],columns=['latitude','longitude'])
# revient à faire :
# myHome=pd.DataFrame(index=range(1))
# myHome["latitude"]=5.720382352069388
# myHome["longitude"]=45.18841421604648
myhomeTuple=(5.720382352069388,45.18841421604648)

In [29]:
#on calcul les distances entre les arbres et chez moi
azerty["coordinates"].apply(lambda y: distance.distance(y,myhomeTuple).meters).argsort()

13604    248
3492     345
23274    470
20823    184
6014      78
        ... 
8965     210
16124    459
8607     388
16842    348
10741    316
Name: coordinates, Length: 500, dtype: int64

In [15]:
# Quelle est la distance moyenne entre les arbres ? 2 méthodes

In [16]:
# méthode 1
#%%timeit
azerty_comboBIS=azerty["coordinates"][0:500].apply(lambda x: azerty["coordinates"][0:500]
                            .transpose().apply(lambda y: distance.distance(x, y).meters))
azerty_comboBIS

,13604,3492,23274,20823,6014,4577,7173,2880,1657,21616,...,2756,28353,314,6420,21826,8965,16124,8607,16842,10741
13604,0.000000,649.988449,3398.520721,1218.682859,3493.077235,2657.454334,3245.511492,2572.186613,4985.339460,5311.696682,...,2357.732861,3271.589210,5211.368550,586.501427,2466.982587,2694.055877,4594.480790,4223.944314,3254.347165,3109.658241
3492,649.988449,0.000000,2909.617107,662.895851,3339.107519,2629.564479,2615.469659,2413.782143,4554.374612,4896.525513,...,2260.420756,2803.199898,4775.891882,499.862170,1911.308707,2274.298007,4137.191737,3699.254228,2756.888436,3182.598235
23274,3398.520721,2909.617107,0.000000,2972.343912,1993.088400,2325.840568,1430.188618,1795.411051,1690.978834,2053.703072,...,2035.870015,185.523887,1900.116027,3396.107861,1068.864950,790.566593,1242.091052,866.410886,161.212165,2917.914320
20823,1218.682859,662.895851,2972.343912,0.000000,3761.592852,3155.410413,2350.201831,2861.745925,4660.723515,5018.879599,...,2756.543046,2902.966937,4872.273199,732.517136,1907.324465,2471.190522,4213.656693,3674.599457,2811.852429,3750.397708
6014,3493.077235,3339.107519,1993.088400,3761.592852,0.000000,974.826052,3289.886298,928.781495,2367.534031,2527.064902,...,1141.797879,1835.001201,2567.057822,3807.136987,2425.657714,1630.405344,2272.014647,2579.435390,2032.127536,1178.382497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8965,2694.055877,2274.298007,790.566593,2471.190522,1630.405344,1664.673579,1698.333285,1122.724268,2292.892186,2626.601848,...,1314.658188,622.273938,2518.092210,2773.222980,803.033408,0.000000,1904.129569,1655.973069,697.352962,2311.242923
16124,4594.480790,4137.191737,1242.091052,4213.656693,2272.014647,2995.661622,2389.246280,2560.606816,477.111474,846.533860,...,2850.458966,1333.999898,664.469860,4628.488416,2307.475962,1904.129569,0.000000,774.013213,1402.020639,3424.599997
8607,4223.944314,3699.254228,866.410886,3674.599457,2579.435390,3098.155333,1653.514535,2591.232910,1232.468409,1590.083132,...,2853.081705,1036.492382,1376.867715,4170.865503,1793.570759,1655.973069,774.013213,0.000000,985.674496,3635.442323
16842,3254.347165,2756.888436,161.212165,2811.852429,2032.127536,2295.527035,1322.581645,1757.870737,1852.153610,2214.894492,...,1983.468704,203.886487,2060.920978,3241.495751,907.707751,697.352962,1402.020639,985.674496,0.000000,2906.499308


In [17]:
# # méthode 2 : avec itertools.combinations pour éviter les doublons 
azerty_combo=[distance.distance(a,b).meters for a, b 
              in combinations(azerty["coordinates"][0:500].tolist(),2)]

# on met en forme les résultats dans un DataFrame et on met en index quels arbres sont 
# comparés entre eux
df_azerty_combo=pd.DataFrame(azerty_combo)
df_azerty_combo["combo"]=list((i,j) for ((i,_),(j,_)) 
                                         in combinations
                                          (enumerate(azerty["coordinates"][0:500].tolist()), 2))
df_azerty_combo.columns=["Distance","Combo"]
df_azerty_combo.set_index("Combo", inplace=True)
df_azerty_combo

,Distance
Combo,
"(0, 1)",649.988449
"(0, 2)",3398.520721
"(0, 3)",1218.682859
"(0, 4)",3493.077235
"(0, 5)",2657.454334
...,...
"(496, 498)",1402.020639
"(496, 499)",3424.599997
"(497, 498)",985.674496


In [18]:
#distance moyenne entre les x (taille sample) arbres de la liste (en mètres) ?
df_azerty_combo.mean()

Distance    2457.077833
dtype: float64

In [19]:
#Quel est l’arbre le plus entouré ?

#on crée un indice de proximité en sommant l'inverse de la distance des arbres proches
df_proximity= azerty_comboBIS[azerty_comboBIS!=0]
df_proximity=1/df_proximity
df_proximity

,13604,3492,23274,20823,6014,4577,7173,2880,1657,21616,...,2756,28353,314,6420,21826,8965,16124,8607,16842,10741
13604,NaN,0.001538,0.000294,0.000821,0.000286,0.000376,0.000308,0.000389,0.000201,0.000188,...,0.000424,0.000306,0.000192,0.001705,0.000405,0.000371,0.000218,0.000237,0.000307,0.000322
3492,0.001538,NaN,0.000344,0.001509,0.000299,0.000380,0.000382,0.000414,0.000220,0.000204,...,0.000442,0.000357,0.000209,0.002001,0.000523,0.000440,0.000242,0.000270,0.000363,0.000314
23274,0.000294,0.000344,NaN,0.000336,0.000502,0.000430,0.000699,0.000557,0.000591,0.000487,...,0.000491,0.005390,0.000526,0.000294,0.000936,0.001265,0.000805,0.001154,0.006203,0.000343
20823,0.000821,0.001509,0.000336,NaN,0.000266,0.000317,0.000425,0.000349,0.000215,0.000199,...,0.000363,0.000344,0.000205,0.001365,0.000524,0.000405,0.000237,0.000272,0.000356,0.000267
6014,0.000286,0.000299,0.000502,0.000266,NaN,0.001026,0.000304,0.001077,0.000422,0.000396,...,0.000876,0.000545,0.000390,0.000263,0.000412,0.000613,0.000440,0.000388,0.000492,0.000849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8965,0.000371,0.000440,0.001265,0.000405,0.000613,0.000601,0.000589,0.000891,0.000436,0.000381,...,0.000761,0.001607,0.000397,0.000361,0.001245,NaN,0.000525,0.000604,0.001434,0.000433
16124,0.000218,0.000242,0.000805,0.000237,0.000440,0.000334,0.000419,0.000391,0.002096,0.001181,...,0.000351,0.000750,0.001505,0.000216,0.000433,0.000525,NaN,0.001292,0.000713,0.000292
8607,0.000237,0.000270,0.001154,0.000272,0.000388,0.000323,0.000605,0.000386,0.000811,0.000629,...,0.000350,0.000965,0.000726,0.000240,0.000558,0.000604,0.001292,NaN,0.001015,0.000275
16842,0.000307,0.000363,0.006203,0.000356,0.000492,0.000436,0.000756,0.000569,0.000540,0.000451,...,0.000504,0.004905,0.000485,0.000308,0.001102,0.001434,0.000713,0.001015,NaN,0.000344


In [20]:
#Quel est l’arbre le plus entouré ? 242 avec indice prox = 2.883
#Quel est l’arbre le plus solitaire ? 75 avec indice prox = 0.170
print(df_proximity.sum().argsort())
print(df_proximity.sum().describe())


13604    316
3492     405
23274    392
20823    474
6014     242
        ... 
8965     279
16124    240
8607     157
16842     85
10741     53
Length: 500, dtype: int64
count    500.000000
mean       0.363681
std        0.107674
min        0.117026
25%        0.300002
50%        0.353687
75%        0.406441
max        1.284621
dtype: float64


In [21]:
# L'arbre le plus entouré est en vert et le plus éloigné est en jaune
i = df_proximity.sum().argsort().iloc[0]
folium.CircleMarker(location=[azerty["coordinates"].iloc[i][1],
                            azerty["coordinates"].iloc[i][0]],
                    radius=4, color= 'yellow').add_to(m)
i = df_proximity.sum().argsort().iloc[499]
folium.CircleMarker(location=[azerty["coordinates"].iloc[i][1],
                            azerty["coordinates"].iloc[i][0]],
                    radius=4, color= 'green').add_to(m)
m

In [22]:
azerty

,type,coordinates
13604,Point,"[5.74557121291162, 45.1883950991829]"
3492,Point,"[5.73984876257375, 45.1897349392367]"
23274,Point,"[5.71755493262181, 45.175784450631]"
20823,Point,"[5.73657286272301, 45.1947469025278]"
6014,Point,"[5.72876386133335, 45.1616942436905]"
...,...,...
8965,Point,"[5.7247036113896, 45.1758442284526]"
16124,Point,"[5.70928006860134, 45.1682021368581]"
8607,Point,"[5.70974403260293, 45.1751742813924]"
16842,Point,"[5.71848743575016, 45.1769031442266]"


In [23]:
# on ajoute une visualisation en Heatmap
from folium.plugins import HeatMap
# on inverse latitude et longitude pour l'entré en paramètre de HeatMap
list_inverse=[]
for i in range(500):
    list_inverse.append([azerty["coordinates"].iloc[i][1],
                            azerty["coordinates"].iloc[i][0]])
HeatMap(list_inverse).add_to(m)
m